In [ ]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        directory = os.path.join(dirname, filename)
print(directory)

In [ ]:
data = pd.read_csv(directory)
print(data.shape)
print(list(data.columns))

In [ ]:
#First column is the target, therefore remove it and change it to a new dataframe
#Keep the data for the training
Targets = data.pop("class")
print(data.shape) #Less one column because of the target
print(Targets.shape) #Only one column (just the target)

In [ ]:
Targets = np.array((Targets))
print(Targets)

In [ ]:
#Function for hot-encoding like: [0,1,0], [1,0,0], [0,0,1]
def hot_encoding(Array):
    dic = {}
    cont = 0
    for j in Array:
        if j not in dic:
            dic[j] = cont
            cont += 1
    if len(dic) == 2: #This is a binary classification
        array = np.zeros((Array.shape[0]))
        for i, j in enumerate(Array):
            array[i] = dic[str(j)]
    else: #For multi-class classification
        array = np.zeros((Array.shape[0],len(dic)))
        for i, j in enumerate(Array):
            array[i,dic[str(j)]] = 1
    return array
#Function for encoding
def encoding(Array):
    array = np.zeros_like(Array)
    for k in range(Array.shape[1]):
        dic = {}
        cont = 0
        for j in Array[:,k]:
            if j not in dic:
                dic[j] = cont
                cont += 1
        for i, j in enumerate(Array[:,k]):
            array[i,k] = dic[j]
    return array
#Hot encoding time!!    
Targets_h = hot_encoding(Targets)
print("Without hot-encoding")
print(Targets)
print("With hot-encoding")
print(Targets_h)

In [ ]:
#Encode the data
data = np.array((data))
data_en = encoding(data)
print(data)
print(data_en)

In [ ]:
print(data_en.shape)
print(Targets_h.shape)

In [ ]:
#let's create our neural network shall we?
def build_nn():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(22,)))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    #model.add(tf.keras.layers.Dropout(0.2))
    #model.add(tf.keras.layers.Dense(64, activation="relu"))
    model.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
    return model

In [ ]:
#Get the model
model = build_nn()
model.summary()

In [ ]:
data_en = np.array((data_en), dtype="int64")

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer="sgd", metrics=["accuracy"])

In [ ]:
history = model.fit(data_en, Targets_h, epochs=100, validation_split=0.1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()